In [1]:
file = "/Users/ryuto/lab/research/work/ACL2020/train.jsonl"

In [2]:
import json
from itertools import islice

def read_file(file):
    with open(file) as fi:
        for line in fi:
            line = line.strip()
            if not line:
                continue
            yield json.loads(line)

In [75]:
def set_vocab(vocab_file):
    """Open file of pre-trained vocab and convert to dict format."""
    print("\n# Load '{}'".format(vocab_file))
    vocab = {}
    f = open(vocab_file)
    for line in f:
        split_line = line.rstrip().split("\t")
        word, idx = split_line[0], split_line[1]
        vocab[word] = idx
    f.close()

    return vocab

In [13]:
from toolz import sliding_window
import random

MASK = "M"

def create_before_before(instance, n_insert=(3, 5)):
    case_converter = {0: "が", 1: "を", 2: "に"}
    phrase_ids = [idx for idx, v in enumerate(instance["bunsetsu"]) if v == 1] + [len(instance["tokens"])]
    phrase_range = [(sta, end) for sta, end in sliding_window(2, phrase_ids)]    
    tree = [p for p in instance["tree"] if p]
    assert len(phrase_range) == len(tree)
    
    print(" ".join("".join(instance["surfaces"][sta:end]) for sta, end in phrase_range))
    
    for pas in instance["pas"]:
        predicate = instance["surfaces"][pas["p_id"]]
        zero_ids = [idx for idx, t in enumerate(pas["types"]) if t == "zero"]
        
        for zero_idx in zero_ids:
            args = [instance["surfaces"][zero_idx], case_converter[pas["args"][zero_idx]], predicate]
            
            # Create Path from zero
            path_zero = {}
            target = None
            for (sta, end), (index, head) in zip(phrase_range, tree):
                if target is None and sta <= zero_idx < end:
                    path_zero[index] = list(range(sta, end))
                    args_idx = index
                    target = head
                if target is not None and target == index:
                    path_zero[index] = list(range(sta, end))
                    target = head

            target = None
            for (sta, end), (index, head) in zip(phrase_range, tree):
                if target is None and sta <= zero_idx < end:
                    path_zero[index] = list(range(sta, end))
                    target = index
                if target is not None and target == index:
                    path_zero[index] = list(range(sta, end))
                    target = index
                    
            # Create Path from predicate
            path_predicate_after = {}
            predicate_idx = None
            target = None
            for (sta, end), (index, head) in zip(phrase_range, tree):
                if target is None and sta <= pas["p_id"] < end:
                    path_predicate_after[index] = list(range(sta, end))
                    predicate_idx = index
                    target = head
                if target is not None and target == index:
                    path_predicate_after[index] = list(range(sta, end))
                    target = head

            path_predicate_before = {}
            target = None
            for (sta, end), (index, head) in zip(phrase_range[::-1], tree[::-1]):
                if target is None and sta <= pas["p_id"] < end:
                    path_predicate_before[index] = list(range(sta, end))
                    target = index
                if target is not None and target == head:
                    path_predicate_before[index] = list(range(sta, end))
                    target = index

            if predicate_idx < args_idx:
                continue
                    
            # Merge path & Create text_a
            end_idx = min(path_zero.keys() & path_predicate_after.keys())
            fake_args_idx = min(path_predicate_before)
            path_zero.update(path_predicate_after)
            path_zero.update(path_predicate_before)
            merged_indices, insert_position, n_words, before_idx = [], [], 0, None
            text_a = [MASK] * random.randint(*n_insert)
            for k in sorted(path_zero):
                if before_idx is None:
                    before_idx = k
                elif k == fake_args_idx:
                    insert_position.append(n_words)
                    text_a += [MASK] * random.randint(*n_insert)
                    text_a += ["、、", instance["surfaces"][zero_idx], case_converter[pas["args"][zero_idx]]]
                elif before_idx + 1 != k:
                    insert_position.append(n_words)
                    text_a += [MASK] * random.randint(*n_insert)
                merged_indices += path_zero[k]
                text_a += [instance["surfaces"][idx] for idx in path_zero[k]]
                n_words += len(path_zero[k])
                if k == end_idx:
                    break
                before_idx = k
            if end_idx != tree[-1][0]:
                text_a += [MASK] * random.randint(*n_insert)
                insert_position.append(n_words)
            
            mask_ids = [idx for idx, surface in enumerate(text_a) if surface == MASK]
                    
            # Create new instance
            keys = ["tokens", "surfaces", "bases", "pos", "bunsetsu", "tree"]
            new_instance = {k: [instance[k][idx] for idx in merged_indices] for k in keys}
            new_instance["insert_position"] = insert_position
            new_instance["text_a"] = text_a
            new_instance["mask_ids"] = mask_ids
            
            yield args, new_instance

In [61]:
for instance in islice(read_file(file), 10):
    for args, instance in create_before_before(instance):
        print(", ".join(args), 
              "\n\t"+"".join(instance["surfaces"]), 
              "\n\t"+"".join(instance["text_a"]).rstrip("。")+"。")

村山富市首相は 年頭に あたり 首相官邸で 内閣記者会と 二十八日 会見し、 社会党の 新民主連合所属議員の 離党問題に ついて 「政権に 影響を 及ぼす ことには ならない。 離党者が いても、 その 範囲に とどまると 思う」と 述べ、 大量離党には 至らないとの 見通しを 示した。
問題, が, 及ぼす 
	離党問題について影響を及ぼすことにはならない。思う」と述べ、 
	MMMMM離党問題についてMMMMM、、問題が影響を及ぼすことにはならない。MMMM思う」と述べ、MMM。
問題, が, なら 
	離党問題について影響を及ぼすことにはならない。思う」と述べ、 
	MMM離党問題についてMMMM、、問題が影響を及ぼすことにはならない。MMM思う」と述べ、MMMMM。
問題, が, とどまる 
	離党問題についてその範囲にとどまると思う」と述べ、 
	MMMM離党問題についてMMMMM、、問題がその範囲にとどまると思う」と述べ、MMMM。
首相, が, 思う 
	村山富市首相は会見し、その範囲にとどまると思う」と述べ、示した。 
	MMM村山富市首相はMMMM会見し、MMM、、首相がその範囲にとどまると思う」と述べ、MMMMM示した。
首相, が, 述べ 
	村山富市首相は会見し、その範囲にとどまると思う」と述べ、示した。 
	MMMMM村山富市首相はMMMM会見し、MMM、、首相がその範囲にとどまると思う」と述べ、MMMM示した。
問題, が, 至ら 
	離党問題について述べ、大量離党には至らないとの見通しを示した。 
	MMM離党問題についてMMMM述べ、MMMM、、問題が大量離党には至らないとの見通しを示した。
首相, が, 示した 
	村山富市首相は会見し、大量離党には至らないとの見通しを示した。 
	MMMMM村山富市首相はMMM会見し、MMMM、、首相が大量離党には至らないとの見通しを示した。
また、 一九九五年中の 衆院解散・総選挙の 可能性に 否定的な 見解を 表明、 二十日 召集予定の 通常国会前の 内閣改造を 明確に 否定した。
ロシア南部チェチェン共和国の 首都グロズヌイに 進攻した ロシア軍は 三十一日、 首都中心部を 装甲車などで 攻撃、 大統領官邸など 数カ所が 炎上した。
ロシア側は 首都制圧の 最終 段階に 入ったと みられる。

In [64]:
from toolz import sliding_window
import random

MASK = "M"

def create_before_after(instance, n_insert=(3, 5)):
    case_converter = {0: "が", 1: "を", 2: "に"}
    phrase_ids = [idx for idx, v in enumerate(instance["bunsetsu"]) if v == 1] + [len(instance["tokens"])]
    phrase_range = [(sta, end) for sta, end in sliding_window(2, phrase_ids)]    
    tree = [p for p in instance["tree"] if p]
    assert len(phrase_range) == len(tree)
    
    print(" ".join("".join(instance["surfaces"][sta:end]) for sta, end in phrase_range))
    
    for pas in instance["pas"]:
        predicate = instance["surfaces"][pas["p_id"]]
        zero_ids = [idx for idx, t in enumerate(pas["types"]) if t == "zero"]
        
        for zero_idx in zero_ids:
            args = [instance["surfaces"][zero_idx], case_converter[pas["args"][zero_idx]], predicate]
            
            # Create Path from zero
            args_idx = None
            for (sta, end), (index, head) in zip(phrase_range, tree):
                if sta <= zero_idx < end:
                    args_idx = index
                    break

            predicate_idx = None
            path_predicate_before = {}
            target = None
            for (sta, end), (index, head) in zip(phrase_range[::-1], tree[::-1]):
                if target is None and sta <= pas["p_id"] < end:
                    path_predicate_before[index] = list(range(sta, end))
                    predicate_idx = index
                    target = index
                if target is not None and target == head:
                    path_predicate_before[index] = list(range(sta, end))
                    target = index

            if predicate_idx < args_idx:
                continue
                    
            merged_indices = []
            text_a = []
            for k in sorted(path_predicate_before):
                if k == predicate_idx:
                    for idx in path_predicate_before[k]:
                        if idx == pas["p_id"]:
                            text_a += [instance["bases"][idx], "、"]
                            merged_indices.append(idx)
                            break
                        else:
                            text_a += [instance["surfaces"][idx]]
                            merged_indices.append(idx)
                    break
                else:
                    merged_indices += path_predicate_before[k]
                    text_a += [instance["surfaces"][idx] for idx in path_predicate_before[k]]
            n =  random.randint(*n_insert)
            length = len(merged_indices)
            insert_position = [length]
            mask_ids = list(range(length, length + n))
            text_a += [MASK] * n
            text_a += [instance["surfaces"][zero_idx]]
                    
            # Create new instance
            keys = ["tokens", "surfaces", "bases", "pos", "bunsetsu", "tree"]
            new_instance = {k: [instance[k][idx] for idx in merged_indices] for k in keys}
            new_instance["insert_position"] = insert_position
            new_instance["text_a"] = text_a
            new_instance["mask_ids"] = mask_ids
            
            yield args, new_instance

In [119]:
PARTICLE = ["が", "の", "を", "に", "へ", "と", "より", "から", "で"]

for instance in islice(read_file(file), 10):
    for args, instance in create_before_after(instance):
        print(", ".join(args), 
              "\n\t"+"".join(instance["surfaces"]), 
              "\n\t"+"".join(instance["text_a"]),
              "\n\t" + random.choice(PARTICLE) + random.randint(3, 5) * MASK + "。")

村山富市首相は 年頭に あたり 首相官邸で 内閣記者会と 二十八日 会見し、 社会党の 新民主連合所属議員の 離党問題に ついて 「政権に 影響を 及ぼす ことには ならない。 離党者が いても、 その 範囲に とどまると 思う」と 述べ、 大量離党には 至らないとの 見通しを 示した。
問題, が, 及ぼす 
	影響を及ぼす 
	影響を及ぼす、MMMM問題 
	からMMMM。
問題, が, なら 
	影響を及ぼすことにはなら 
	影響を及ぼすことにはなる、MMM問題 
	をMMMM。
問題, が, とどまる 
	その範囲にとどまる 
	その範囲にとどまる、MMMM問題 
	でMMMM。
首相, が, 思う 
	その範囲にとどまると思う 
	その範囲にとどまると思う、MMMM首相 
	とMMMMM。
首相, が, 述べ 
	その範囲にとどまると思う」と述べ 
	その範囲にとどまると思う」と述べる、MMMMM首相 
	にMMM。
問題, が, 至ら 
	大量離党には至ら 
	大量離党には至る、MMM問題 
	でMMMM。
首相, が, 示した 
	大量離党には至らないとの見通しを示した 
	大量離党には至らないとの見通しを示す、MMM首相 
	にMMMMM。
また、 一九九五年中の 衆院解散・総選挙の 可能性に 否定的な 見解を 表明、 二十日 召集予定の 通常国会前の 内閣改造を 明確に 否定した。
ロシア南部チェチェン共和国の 首都グロズヌイに 進攻した ロシア軍は 三十一日、 首都中心部を 装甲車などで 攻撃、 大統領官邸など 数カ所が 炎上した。
ロシア側は 首都制圧の 最終 段階に 入ったと みられる。
制圧, が, 入った 
	最終段階に入った 
	最終段階に入る、MMMM制圧 
	とMMM。
グロズヌイからの 報道では、 ロシア軍は 激しい 空爆と 砲撃を 加えた 後、 装甲車部隊が 大統領官邸付近に 進出。
軍, が, 加えた 
	空爆と砲撃を加えた 
	空爆と砲撃を加える、MMMMM軍 
	よりMMMM。
同官邸前などで ドゥダエフ政権部隊と 激しい 市街戦を 展開している。
一方、 ドゥダエフ政権側の 首都防衛司令官は 同日 夕、 テレビを 通じ、 首都防衛は うまく いっており、 ロシア軍の 戦車 五十両を 破壊したと 発表。
官, が, 通じ 

In [109]:
def extract_paths(instance, pas):
    """
    Yields:
        Paths： (項の前，項の後ろ，述語の前，述語の後ろ)
        triple: (target_word, particle, predicate)
        position_type： 'before' or 'after'
    """
    case_converter = {0: "が", 1: "を", 2: "に"}
    phrase_ids = [idx for idx, v in enumerate(instance["bunsetsu"]) if v == 1] + [len(instance["tokens"])]
    phrase_range = [(sta, end) for sta, end in sliding_window(2, phrase_ids)]
    tree = [p for p in instance["tree"] if p]
    assert len(phrase_range) == len(tree)

    predicate = instance["surfaces"][pas["p_id"]]
    zero_ids = [idx for idx, t in enumerate(pas["types"]) if t == "zero"]
    for zero_idx in zero_ids:
        path_zero_before, path_zero_after, path_predicate_before, path_predicate_after,  = {}, {}, {}, {}
        predicate_idx, arg_idx = None, None

        # Create Path from zero (after)
        target = None
        for (sta, end), (index, head) in zip(phrase_range, tree):
            if target is None and sta <= zero_idx < end:
                path_zero_after[index] = list(range(sta, end))
                arg_idx = index
                target = head
            elif target is not None and target == index:
                path_zero_after[index] = list(range(sta, end))
                target = head

        # Create Path from zero (before)
        target = None
        for (sta, end), (index, head) in zip(phrase_range[::-1], tree[::-1]):
            if target is None and sta <= zero_idx < end:
                path_zero_before[index] = list(range(sta, end))
                target = index
            elif target is not None and target == head:
                path_zero_before[index] = list(range(sta, end))
                target = index

        # Create Path from predicate (after)
        target = None
        for (sta, end), (index, head) in zip(phrase_range, tree):
            if target is None and sta <= pas["p_id"] < end:
                path_predicate_after[index] = list(range(sta, end))
                predicate_idx = index
                target = head
            if target is not None and target == index:
                path_predicate_after[index] = list(range(sta, end))
                target = head

        # Create Path from predicate (before)
        target = None
        for (sta, end), (index, head) in zip(phrase_range[::-1], tree[::-1]):
            if target is None and sta <= pas["p_id"] < end:
                path_predicate_before[index] = list(range(sta, end))
                target = index
            if target is not None and target == head:
                path_predicate_before[index] = list(range(sta, end))
                target = index

        paths = (path_zero_before, path_zero_after, path_predicate_before, path_predicate_after)
        triple = (instance["surfaces"][zero_idx], case_converter[pas["args"][zero_idx]], predicate)
        position_type = "before" if arg_idx < predicate_idx else "after"

        yield paths, triple, position_type

In [124]:
def beta_b(instance, pas, paths, triple, vocab, n_insert=(3, 5)):
    """β-b：「項 > 述語」 -> 「項 > 述語」へ 変換"""
    target_word, particle, predicate = triple

    # Merge path
    path_zero_before, _, path_predicate_before, _ = paths

    # Create Instance
    before_idx = 0
    indices, insert_position = [], {}
    tree = [t for t in instance["tree"] if t is not None]
    text_a = []

    for k in sorted(path_zero_before):
        if k <= max(path_predicate_before):
            continue
        
        # Insert MASK
        if before_idx + 1 != k:
            n = random.randint(*n_insert)
            insert_position[len(indices)] = n
            text_a += [MASK] * n
        indices += path_zero_before[k]
        text_a += [instance["surfaces"][idx] for idx in path_zero_before[k]]
        before_idx = k

    # Insert MASK
    n = random.randint(*n_insert)
    insert_position[len(indices)] = n
    text_a += [MASK] * n + ["、", "、", target_word, particle]
    indices.append("、")

    for k in sorted(path_predicate_before):
        indices += path_predicate_before[k]
        text_a += [instance["surfaces"][idx] for idx in path_predicate_before[k]]
    # Insert MASK
    if max(path_predicate_before) != tree[-1][0]:
        n = random.randint(*n_insert)
        insert_position[len(indices)] = n
        text_a += [MASK] * n + ["。"]
        indices.append("。")

    tokens = [vocab[i] if type(i) == str else instance["tokens"][i] for i in indices]
    p_ids = [1 if idx == pas["p_id"] else 0 for idx in range(len(instance["tokens"]))]
    p_id = [0 if type(i) == str else p_ids[i] for i in indices].index(1)
    args = [3 if type(i) == str else pas["args"][i] for i in indices]
    instance = {"tokens": tokens,
                "pas": {"p_id": p_id, "args": args},
                "insert_position": insert_position,
                "text_a": text_a,
                "black_list": triple}

    return instance

In [125]:
CASE_PARTICLES = ["が", "の", "を", "に", "へ", "と", "より", "から", "で"]
def alpha_b(instance, pas, paths, triple, vocab, n_insert=(3, 5)):
    """α-b：「項 < 述語」 -> 「項 > 述語」へ 変換 (が格のみ)"""
    target_word, particle, predicate = triple

    # Merge path
    _, _, path_predicate_before, _ = paths

    # Create Instance
    predicate_idx = max(path_predicate_before)
    text_a, text_b, indices, insert_position = [], [], [], {}
    tree = [t for t in instance["tree"] if t is not None]

    for k in sorted(path_predicate_before):
        if k == predicate_idx:
            for idx in path_predicate_before[k]:
                if idx == pas["p_id"]:
                    text_a += [instance["bases"][idx], "、"]
                    indices += [idx, "、"]
                    break
                else:
                    text_a += [instance["surfaces"][idx]]
                    indices.append(idx)
            break
        else:
            indices += path_predicate_before[k]
            text_a += [instance["surfaces"][idx] for idx in path_predicate_before[k]]
    # Insert MASK (text_a)
    n = random.randint(*n_insert)
    insert_position[len(indices)] = n
    text_a += [MASK] * n + [target_word]
    indices.append(target_word)
    # Insert MASK (text_b)
    insert_case_particle = random.choice(CASE_PARTICLES)
    indices.append(insert_case_particle)

    n = random.randint(*n_insert)
    insert_position[len(indices)] = n
    text_b = [insert_case_particle] + [MASK] * n + ["。"]
    indices.append("。")

    # Create new instance
    tokens = [vocab[i] if type(i) == str else instance["tokens"][i] for i in indices]
    p_ids = [1 if idx == pas["p_id"] else 0 for idx in range(len(instance["tokens"]))]
    p_id = [0 if type(i) == str else p_ids[i] for i in indices].index(1)
    args = [3 if type(i) == str else pas["args"][i] for i in indices]
    case_converter = {"が": 0, "を": 1, "に": 2}
    args[-3] = case_converter[particle]
    instance = {"tokens": tokens,
                "pas": {"p_id": p_id, "args": args},
                "insert_position": insert_position,
                "text_a": text_a,
                "text_b": text_b,
                "black_list": [target_word, particle]}

    return instance

In [79]:
vocab = set_vocab("/Users/ryuto/lab/research/data/raw/NTC_Matsu_original/wordIndex.txt")


# Load '/Users/ryuto/lab/research/data/raw/NTC_Matsu_original/wordIndex.txt'


In [128]:
for instance in islice(read_file(file), 200):
    for pas in instance["pas"]:
        for paths, triple, position_type in extract_paths(instance, pas):
            if position_type == "after":
                new_instance = beta_b(instance, pas, paths, triple, vocab)
                print("【beta】")
                print(",".join(new_instance["black_list"]))
                print("".join(new_instance["text_a"]))
                print("".join(instance["surfaces"]))

                
                new_instance2 = alpha_b(instance, pas, paths, triple, vocab)
                print("【alpha】")
                print(",".join(new_instance2["black_list"]))
                print("".join(new_instance2["text_a"]))
                print("".join(new_instance2["text_b"]))
                print("".join(instance["surfaces"]))
                print()

【beta】
首都,に,した
MMMM首都中心部をMMMMM、、首都にロシア南部チェチェン共和国の首都グロズヌイに進攻したMMMMM。
ロシア南部チェチェン共和国の首都グロズヌイに進攻したロシア軍は三十一日、首都中心部を装甲車などで攻撃、大統領官邸など数カ所が炎上した。
【alpha】
首都,に
ロシア南部チェチェン共和国の首都グロズヌイに進攻する、MMMM首都
へMMMMM。
ロシア南部チェチェン共和国の首都グロズヌイに進攻したロシア軍は三十一日、首都中心部を装甲車などで攻撃、大統領官邸など数カ所が炎上した。

【beta】
政党,が,立った
MMMMM平和と民主主義を担う政党がMMMM、、政党が市民の側に立ったMMMMM。
党内の議論や党関係者の意見は「保守二党論はよろしくない。市民の側に立った平和と民主主義を担う政党が必要」というものだ。
【alpha】
政党,が
市民の側に立つ、MMM政党
がMMMM。
党内の議論や党関係者の意見は「保守二党論はよろしくない。市民の側に立った平和と民主主義を担う政党が必要」というものだ。

【beta】
米,が,する
MMM今度の日米首脳会談のMMM、、米が二十一世紀を展望するMMM。
戦後五十年の節目であり、冷戦構造が崩壊し、世界が新秩序を求めている時期に二十一世紀を展望する日米の親密な関係をどう構築するかが今度の日米首脳会談の大きな課題だ。
【alpha】
米,が
二十一世紀を展望する、MMM米
にMMM。
戦後五十年の節目であり、冷戦構造が崩壊し、世界が新秩序を求めている時期に二十一世紀を展望する日米の親密な関係をどう構築するかが今度の日米首脳会談の大きな課題だ。

【beta】
米,が,する
MMM今度の日米首脳会談のMMM、、米がどう構築するかがMMMMM。
戦後五十年の節目であり、冷戦構造が崩壊し、世界が新秩序を求めている時期に二十一世紀を展望する日米の親密な関係をどう構築するかが今度の日米首脳会談の大きな課題だ。
【alpha】
米,が
どう構築する、MMM米
とMMMM。
戦後五十年の節目であり、冷戦構造が崩壊し、世界が新秩序を求めている時期に二十一世紀を展望する日米の親密な関係をどう構築するかが今度の日米首脳会談の大きな課題だ。

【beta】
日本,が,乗り切る
MMM「日本はMMMM、、日本が混迷の転換

KeyError: 'ＡＰＥＣ'